### Open Flights Data Wrangling

To practice, you are going to wrangle data from OpenFlights.  You can read about it here: 

http://openflights.org/data.html

This includes five files: 

1. A file with one record for each airport, 
2. A file with one record for each airline, 
3. A file with one record for each each route, 
4. A file with one record for each type of airplane, and
5. A file with one record for each country.  

Each file has a relationship to the others, with some fields in common.  To make best use of the data, we will need to combine all three files.  This is the core challenge of relational databases.  This lesson looks at how to do that, specifically using the merge() function in python.  

Credits:

This lesson draws from the Data Analysis and Visualization in Python for Ecologists course by Data Carpentry:

https://datacarpentry.org/python-ecology-lesson/05-merging-data/index.html

and from the pandas documentation:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html


### Learning Objectives: 

1. Understand the meaning of relational data and be able to identify common keys. 

2. Understand the different types of relational joins: concatenate, inner join, outer join, left join, right join. 

3. Use the pandas syntax pd.concat() and pd.merge() to perform those relational joins. 

4. Start to build an awareness of open data resources, and of documentation/tutorials for ongoing learning. 

In [2]:
import pandas as pd
import numpy as np

In [5]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["ID", "NAME", "CITY", "COUNTRY", "IATA", "ICAO", "LATITUDE", "LONGITUDE", "ALTITUDE", "TIMEZONE", "DST", "TZ", "TYPE", "SOURCE"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["ID", "NAME", "ALIAS", "IATA", "ICAO", "CALLSIGN", "COUNTRY", "ACTIVE"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["AIRLINE", "AIRLINE_ID", "SOURCE", "SOURCE_ID", "DEST", "DEST_ID", "CODESHARE", "STOPS", "EQUIPMENT"]

# Read in the planes data.
planes = pd.read_csv("data/planes.dat", header=None, na_values='\\N')
planes.columns = ["NAME", "IATA", "ICAO"]

# Read in the countries data.
countries = pd.read_csv("data/countries.dat", header=None, na_values='\\N')
countries.columns = ["NAME", "ISO", "DAFIF"]

### Start by seeing what's in the data.  

What columns are there?  What data types are the columns?  

Remember, 'object' means it is a string, while the numerical values can be floats or ints.  Sometimes you will have problems if it reads numeric data in as strings.  If that happens, you can use the function .astype() to convert it.  Look it up in the pandas API to get more details

Start with the airports table

In [6]:
# printing the first few rows gives us a good sense of what is in there
airports.head()

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,airport,OurAirports


In [7]:
# we can also look a the data types
# alternatively, we could summarize the data as we did in the previous lesson
airports.dtypes

ID             int64
NAME          object
CITY          object
COUNTRY       object
IATA          object
ICAO          object
LATITUDE     float64
LONGITUDE    float64
ALTITUDE       int64
TIMEZONE     float64
DST           object
TZ            object
TYPE          object
SOURCE        object
dtype: object

##### Your turn:  

Spend a few minutes examining the remaining tables.  See if you can identify which fields are common across multiple tables.  These are the "keys".  Together, we will make sure we have identified the right keys. 

In [8]:
airlines.head()

,ID,NAME,ALIAS,IATA,ICAO,CALLSIGN,COUNTRY,ACTIVE
0,-1,Unknown,NaN,-,NaN,NaN,NaN,Y
1,1,Private flight,NaN,-,NaN,NaN,NaN,Y
2,2,135 Airways,NaN,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,NaN,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,NaN,NaN,WYT,NaN,United Kingdom,N


In [9]:
routes.head()

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
0,2B,410.0,AER,2965.0,KZN,2990.0,NaN,0,CR2
1,2B,410.0,ASF,2966.0,KZN,2990.0,NaN,0,CR2
2,2B,410.0,ASF,2966.0,MRV,2962.0,NaN,0,CR2
3,2B,410.0,CEK,2968.0,KZN,2990.0,NaN,0,CR2
4,2B,410.0,CEK,2968.0,OVB,4078.0,NaN,0,CR2


In [10]:
planes.head()

,NAME,IATA,ICAO
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45


### Selecting data

We might want to work with a subset of data.  If so, we can select data just like we did in the previous lesson. 

In [11]:
# we can select all airports in one of the least visited countries on earth
airports[airports['COUNTRY']=="Tuvalu"]

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
3835,4077,Funafuti International Airport,Funafuti,Tuvalu,FUN,NGFU,-8.525,179.195999,9,12.0,U,Pacific/Funafuti,airport,OurAirports


In [12]:
# we can select all routes to that airport
routes[routes['DEST']=='FUN']

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5


In [13]:
# and routes from that airport
routes[routes['SOURCE']=='FUN']

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [14]:
# where are these flights going to/from? 


#####  Your turn: 

Select just the routes that go to or from Lexington Bluegrass Airport, and store them in their own dataframe.  

The airport code is LEX.  You should have a much smaller dataframe.  How many inbound routes and how many outbound routes are there? 

In [15]:
airports[airports["IATA"]=="LEX"]

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
3782,4017,Blue Grass Airport,Lexington KY,United States,LEX,KLEX,38.036499,-84.605904,979,-5.0,A,America/New_York,airport,OurAirports


In [16]:
lex_routes = routes[(routes['SOURCE']=='LEX') | (routes['DEST']=='LEX')]   #flights in and out of lexington
lex_routes

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9


In [17]:
Into_lex_routes = routes[(routes['SOURCE']=='LEX')]   #flights into lexington
Into_lex_routes
len(Into_lex_routes)  #20 routes coming into LEX

20

In [18]:
OutOf_lex_routes = routes[(routes['DEST']=='LEX')]   #flights leaving lexington
OutOf_lex_routes
len(OutOf_lex_routes)  #20 routes coming into LEX

20

### Concatentating data

To concatenate is to link things together in a series or chain.  It is similar to, but more powerful than appending data, because it can work on either rows or columns, is faster, and will perform set logic (union or intersection).  

In [19]:
# let's go back and contatenate the routes both to and from Tuvalu
to_tuvalu = routes[routes['DEST']=='FUN']
from_tuvalu = routes[routes['SOURCE']=='FUN']


In [20]:
# the basic function, given a list of dataframes to concatenate
# note that we call concatenate as a "static" method, rather than a method 
# associated with a specific object. 

pd.concat([to_tuvalu, from_tuvalu])

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [21]:
# if we want to use the dataframe later, we need to assign it to a variable name

tuvalu_routes = pd.concat([to_tuvalu, from_tuvalu])

Note to see more about this method, we can examine the pandas docs:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [22]:
# note that one of the arguments is axis.  
# axis=0 indicates that the rows should be concatenated, 
# while axis=1 indicates to concatenate the columns 

pd.concat([to_tuvalu, from_tuvalu], axis=1)

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0.0,AT5
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Your turn:

Can you use the concat() function to create one dataframe with flights both into and out of LEX?

### Merging data

Sometimes we want to merge, or join, the data instead of concatenating it.  For example, maybe we hate flying on small planes, so want to know what kind of aircraft we would need to fly on to visit Tuvalu.  This data comes from a different table. 

In [23]:
# we see that the Tuvalu routes operate with AT5 equipment
tuvalu_routes

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [24]:
# but what is an AT5? Let's look in the planes table. 
planes.head()

,NAME,IATA,ICAO
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [25]:
# in this case, we are lucky, and it is in the first few rows.  But to do this systemactically, we
# would want to join the tables.  To do this, we use the merge() function in pandas.  

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


Let's stop and look at what is going on in the above command. 

It is important to understand the type of merge, as specified by the how option.  Here is what the pandas docs say:


![alt text](img/how.png)


![alt text](img/how2.png)

So it looks like the above merge defaulted to an inner join


![alt text](img/inner.png)

In [26]:
# we can check this by explicitly setting the option and confirming that we get the same result  
# next, let's check what happens if we choose the other options

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='inner')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [27]:
# outer join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='outer')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Nord) 262,ND2,N262
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale SN.601 Corvette,NDC,S601
...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-134,TU3,T134
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-154,TU5,T154
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-204,T20,T204
245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yakovlev Yak-40,YK4,YK40


In [28]:
# left join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='left')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [29]:
# right join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='right')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Nord) 262,ND2,N262
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale SN.601 Corvette,NDC,S601
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-134,TU3,T134
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-154,TU5,T154
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-204,T20,T204
245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yakovlev Yak-40,YK4,YK40


I find that I most often want a left join, as described here: 

![alt text](img/left.png)

It may be worth examining some of the other options used in merge().  We can refer to the pandas documentation to learn more. 

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging

#### Your turn

Now let's look at which airlines operate in and out of Lexington.  How many routes does each airline have?  

First, let's spend some time planning together.  Then individually take some time to implement the code we planned. 

#### It looks like there are some international airlines with Lexington routes.  

To look at how many routes they have, create a new column in your dataframe called 'International', which is set to True for an overseas airline and False for a domestic airline.  Calculate the percent of routes with an overseas airline.  

You should have the tools you need to answer this question using what we learned in last week's lesson.  Again, let's start by planning together, then implement the code on your own.  

#### Codeshares

It looks like a bunch of these routes are codeshares.  That means they are marketed by this airline, but operated by a different airline.  See the note in the data documentation on openflights.org/data.  The implication of this is that there are duplicates.

Can you figure out which ones are duplicates?  Can you then create a dataframe with only the unique routes?  How many unique inbound and outbound routes are there? 

Remember, someone has to operate the flight, so if all the routes to/from a particular airport are listed as codeshares, then something is funny...

It is also possible that more than one airline actually operates a route between the same two airports. (Having this sort of competition generally means that you will get better fares as a traveler.)  It may not be obvious what is actually in the data set, so dig or do external research as needed.  

It is this sort of messiness that will consume much of your effort in research and data science.  It is very important to "become one with your data". 

Figuring this out will be the first part of your homework.  

### Review Questions: 

1. What is a key in relational data? 

2. Explain the difference between a left join and an outer join.  

3. Explain the difference between pd.concat() and pd.merge()

4. Identify two possible places to look for help if you get stuck on the homework. 

### Homework 1 - Code shares

Dig into the data as described above, and do your best to identify which routes are code shares, and how many unique routes operate into/out of Lexington.

In [30]:
#create a dataframe with lex in source or destination and codeshares has yes (written Y)
lex_routes_CS = routes[(routes['CODESHARE']=='Y') & (routes['SOURCE']=='LEX') | (routes['CODESHARE']=='Y') & (routes['DEST']=='LEX')] 
lex_routes_CS

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ
21096,DL,2009.0,LEX,4017.0,DCA,3520.0,Y,0,CRJ
21097,DL,2009.0,LEX,4017.0,DTW,3645.0,Y,0,CR7 CRJ CR9


In [31]:
lex_routes["SOURCE"].unique()

array(['ATL', 'LEX', 'CLT', 'DFW', 'ORD', 'DCA', 'DTW', 'LGA', 'MSP',
       'FLL', 'PGD', 'PIE', 'SFB', 'IAH'], dtype=object)

In [32]:
lex_routes["DEST"].unique()

array(['LEX', 'ATL', 'CLT', 'DFW', 'ORD', 'DCA', 'DTW', 'LGA', 'MSP',
       'FLL', 'PGD', 'PIE', 'SFB', 'IAH'], dtype=object)

In [33]:
lex_routes[lex_routes["SOURCE"]=="LEX"]

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9
21095,DL,2009.0,LEX,4017.0,ATL,3682.0,NaN,0,M88 717
21096,DL,2009.0,LEX,4017.0,DCA,3520.0,Y,0,CRJ
21097,DL,2009.0,LEX,4017.0,DTW,3645.0,Y,0,CR7 CRJ CR9
21098,DL,2009.0,LEX,4017.0,LGA,3697.0,NaN,0,ERJ
21099,DL,2009.0,LEX,4017.0,MSP,3858.0,Y,0,CRJ


In [34]:
x=routes[routes["STOPS"]==1]
x

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
2066,5T,1623.0,YRT,132.0,YEK,50.0,NaN,1,ATR
7810,AC,330.0,ABJ,253.0,BRU,302.0,NaN,1,333
8272,AC,330.0,YVR,156.0,YBL,30.0,NaN,1,BEH
17767,CU,1936.0,FCO,1555.0,HAV,1909.0,NaN,1,767
25116,FL,1316.0,HOU,3566.0,SAT,3621.0,NaN,1,735
25220,FL,1316.0,MCO,3878.0,HOU,3566.0,NaN,1,73W
25230,FL,1316.0,MCO,3878.0,ORF,3611.0,NaN,1,717
49746,SK,4319.0,ARN,737.0,GEV,715.0,NaN,1,ATP
63567,WN,4547.0,BOS,3448.0,MCO,3878.0,NaN,1,73W
64042,WN,4547.0,MCO,3878.0,BOS,3448.0,NaN,1,73W


### Homework 2 - The Kevin Bacon Game

Of course, it is possible to fly beyond those few cities from Lexington--you just have to transfer to get there.  Your task is to:

1. Identify which airports you can get to from Lexington with only one transfer.  
2. Identify which airports you can get to from Lexington with two transfers.

You have the tools to do this using the merge functionality presented today, but you will need to be smart about what you are merging.  

Next class, we will use these data to make interactive maps of these flight routes, similar to what you see in the back of the airline magazines.  


#### Bonus: 

This is the air travel version of the Kevin Bacon game (https://oracleofbacon.org/). What is the number N, such that you can reach every airport in the world with N or fewer transfers?

#### Extra Bonus: 

Use this very important piece of knowledge to impress your friends at parties!

In [35]:
len(routes["DEST"].unique())

3418

In [36]:
lex_onetransfer = routes[(routes['SOURCE']=='LEX')]
lex_onetransfer

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9
21095,DL,2009.0,LEX,4017.0,ATL,3682.0,NaN,0,M88 717
21096,DL,2009.0,LEX,4017.0,DCA,3520.0,Y,0,CRJ
21097,DL,2009.0,LEX,4017.0,DTW,3645.0,Y,0,CR7 CRJ CR9
21098,DL,2009.0,LEX,4017.0,LGA,3697.0,NaN,0,ERJ
21099,DL,2009.0,LEX,4017.0,MSP,3858.0,Y,0,CRJ


In [37]:
DEST1=lex_onetransfer["DEST"].unique()
DEST1

array(['ATL', 'CLT', 'DFW', 'ORD', 'DCA', 'DTW', 'LGA', 'MSP', 'FLL',
       'PGD', 'PIE', 'SFB', 'IAH'], dtype=object)

In [93]:
lex_onetransfer_merged=pd.merge(lex_onetransfer, airports, left_on='SOURCE_ID', right_on='ID', how='left')
lex_onetransfer_merged=pd.merge(lex_onetransfer_merged, airports, left_on='DEST_ID', right_on='ID', how='left')
lex_onetransfer_merged=lex_onetransfer_merged.drop_duplicates(subset=['SOURCE_x', 'DEST'], keep='first')
lex_onetransfer_merged


,AIRLINE,AIRLINE_ID,SOURCE_x,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,ID_x,...,IATA_y,ICAO_y,LATITUDE_y,LONGITUDE_y,ALTITUDE_y,TIMEZONE_y,DST_y,TZ_y,TYPE_y,SOURCE
0,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,4017,...,ATL,KATL,33.636700,-84.428101,1026,-5.0,A,America/New_York,airport,OurAirports
1,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ,4017,...,CLT,KCLT,35.214001,-80.943100,748,-5.0,A,America/New_York,airport,OurAirports
2,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4,4017,...,DFW,KDFW,32.896801,-97.038002,607,-6.0,A,America/Chicago,airport,OurAirports
3,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4,4017,...,ORD,KORD,41.978600,-87.904800,672,-6.0,A,America/Chicago,airport,OurAirports
6,DL,2009.0,LEX,4017.0,DCA,3520.0,Y,0,CRJ,4017,...,DCA,KDCA,38.852100,-77.037697,15,-5.0,A,America/New_York,airport,OurAirports
7,DL,2009.0,LEX,4017.0,DTW,3645.0,Y,0,CR7 CRJ CR9,4017,...,DTW,KDTW,42.212399,-83.353401,645,-5.0,A,America/New_York,airport,OurAirports
8,DL,2009.0,LEX,4017.0,LGA,3697.0,NaN,0,ERJ,4017,...,LGA,KLGA,40.777199,-73.872597,21,-5.0,A,America/New_York,airport,OurAirports
9,DL,2009.0,LEX,4017.0,MSP,3858.0,Y,0,CRJ,4017,...,MSP,KMSP,44.882000,-93.221802,841,-6.0,A,America/Chicago,airport,OurAirports
10,G4,35.0,LEX,4017.0,FLL,3533.0,NaN,0,M80,4017,...,FLL,KFLL,26.072599,-80.152702,9,-5.0,A,America/New_York,airport,OurAirports
11,G4,35.0,LEX,4017.0,PGD,7056.0,NaN,0,M80,4017,...,PGD,KPGD,26.920200,-81.990501,26,-5.0,A,America/New_York,airport,OurAirports


In [95]:
lex_onetransfer_merged.to_excel("lex_onetransfer_merged.xlsx")

In [67]:
DEST1=lex_onetransfer["DEST"].unique()
DEST1
lex_onetransfer1=lex_onetransfer[0:0]
for i in range(0,len(DEST1)):
    lex_onetransfer1 =lex_onetransfer1.append(routes[(routes['SOURCE']==DEST1[i])])
lex_onetransfer1=lex_onetransfer1.drop_duplicates()
lex_onetransfer1

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
443,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3
444,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3
445,3M,20710.0,ATL,3682.0,MEI,4335.0,NaN,0,SF3
446,3M,20710.0,ATL,3682.0,MSL,5756.0,NaN,0,SF3
447,3M,20710.0,ATL,3682.0,PIB,5759.0,NaN,0,SF3
...,...,...,...,...,...,...,...,...,...
59370,US,5265.0,IAH,3550.0,PHX,3462.0,NaN,0,319 320
59371,US,5265.0,IAH,3550.0,SAL,1892.0,NaN,0,E90
61079,VA,5360.0,IAH,3550.0,ATL,3682.0,Y,0,M88 717 319
61274,VB,10646.0,IAH,3550.0,MTY,1825.0,NaN,0,733


In [68]:
DEST2=(lex_onetransfer1["DEST"].unique())
DEST2

array(['LWB', 'MCN', 'MEI', 'MSL', 'PIB', 'TUP', 'AZO', 'CHA', 'CID',
       'CRW', 'CVG', 'EVV', 'FWA', 'GSO', 'GSP', 'LAN', 'LEX', 'MBS',
       'MSP', 'ROA', 'SYR', 'TYS', 'XNA', 'CLT', 'DFW', 'LHR', 'MIA',
       'ORD', 'PHL', 'PHX', 'DEN', 'YYZ', 'ABE', 'ABQ', 'ABY', 'AEX',
       'AGS', 'ALB', 'ATW', 'AUS', 'AVL', 'AVP', 'BDL', 'BHM', 'BMI',
       'BNA', 'BOS', 'BQK', 'BRU', 'BTR', 'BTV', 'BUF', 'BWI', 'CAE',
       'CAK', 'CDG', 'CHO', 'CHS', 'CLE', 'CMH', 'COS', 'CSG', 'CUN',
       'CZM', 'DAB', 'DAL', 'DAY', 'DCA', 'DHN', 'DSM', 'DTW', 'DUB',
       'DUS', 'ECP', 'ELP', 'EWN', 'EWR', 'EYW', 'FAY', 'FCO', 'FLL',
       'FNT', 'FRA', 'FSM', 'GDL', 'GNV', 'GPT', 'GRB', 'GRK', 'GRR',
       'GTR', 'HPN', 'HSV', 'IAD', 'IAH', 'ICT', 'ILM', 'IND', 'JAN',
       'JAX', 'JFK', 'LAS', 'LAX', 'LFT', 'LGA', 'LIT', 'MAD', 'MAN',
       'MCI', 'MCO', 'MDT', 'MDW', 'MEM', 'MEX', 'MGM', 'MHT', 'MKE',
       'MLB', 'MLI', 'MLU', 'MOB', 'MSN', 'MSY', 'MTY', 'MUC', 'MYR',
       'NAS', 'OAJ',

In [69]:
len(DEST2)

399

In [96]:
lex_onetransfer1_merged=pd.merge(lex_onetransfer1, airports, left_on='SOURCE_ID', right_on='ID', how='left')
lex_onetransfer1_merged=pd.merge(lex_onetransfer1_merged, airports, left_on='DEST_ID', right_on='ID', how='left')
lex_onetransfer1_merged=lex_onetransfer1_merged.drop_duplicates(subset=['SOURCE_x', 'DEST'], keep='first')
lex_onetransfer1_merged

,AIRLINE,AIRLINE_ID,SOURCE_x,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,ID_x,...,IATA_y,ICAO_y,LATITUDE_y,LONGITUDE_y,ALTITUDE_y,TIMEZONE_y,DST_y,TZ_y,TYPE_y,SOURCE
0,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3,3682,...,LWB,KLWB,37.858299,-80.399498,2302.0,-5.0,U,America/New_York,airport,OurAirports
1,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3,3682,...,MCN,KMCN,32.692799,-83.649200,354.0,-5.0,A,America/New_York,airport,OurAirports
2,3M,20710.0,ATL,3682.0,MEI,4335.0,NaN,0,SF3,3682,...,MEI,KMEI,32.332600,-88.751900,297.0,-6.0,A,America/Chicago,airport,OurAirports
3,3M,20710.0,ATL,3682.0,MSL,5756.0,NaN,0,SF3,3682,...,MSL,KMSL,34.745300,-87.610199,551.0,-6.0,A,America/Chicago,airport,OurAirports
4,3M,20710.0,ATL,3682.0,PIB,5759.0,NaN,0,SF3,3682,...,PIB,KPIB,31.467100,-89.337097,298.0,-6.0,A,America/Chicago,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,UA,5209.0,IAH,3550.0,VSA,1853.0,Y,0,CR7,3550,...,VSA,MMVA,17.997000,-92.817398,46.0,-6.0,S,America/Mexico_City,airport,OurAirports
3536,UA,5209.0,IAH,3550.0,XNA,4356.0,Y,0,CRJ ERJ,3550,...,XNA,KXNA,36.281898,-94.306801,1287.0,-6.0,A,America/Chicago,airport,OurAirports
3539,UA,5209.0,IAH,3550.0,YVR,156.0,NaN,0,320 319 739,3550,...,YVR,CYVR,49.193901,-123.183998,14.0,-8.0,A,America/Vancouver,airport,OurAirports
3542,UA,5209.0,IAH,3550.0,ZIH,1856.0,Y,0,ERJ,3550,...,ZIH,MMZH,17.601601,-101.460999,26.0,-6.0,S,America/Mexico_City,airport,OurAirports


In [101]:
lex_onetransfer1_merged.to_excel("lex_onetransfer1_merged.xlsx")

In [98]:
DEST2=(lex_onetransfer1["DEST"].unique())
lex_twotransfer=lex_onetransfer1[0:0]
for i in range(0,len(DEST2)):
    lex_twotransfer =lex_twotransfer.append(routes[(routes['SOURCE']==DEST2[i])])
lex_twotransfer=lex_twotransfer.drop_duplicates()
lex_twotransfer

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
474,3M,20710.0,LWB,6958.0,ATL,3682.0,NaN,0,SF3
57070,UA,5209.0,LWB,6958.0,IAD,3714.0,Y,0,SF3
475,3M,20710.0,MCN,3754.0,ATL,3682.0,NaN,0,SF3
476,3M,20710.0,MCN,3754.0,MCO,3878.0,NaN,0,SF3
484,3M,20710.0,MEI,4335.0,ATL,3682.0,NaN,0,SF3
...,...,...,...,...,...,...,...,...,...
11276,AM,321.0,ZLO,1858.0,MEX,1824.0,Y,0,E75 ERJ
12048,AS,439.0,ZLO,1858.0,LAX,3484.0,NaN,0,73H
57927,UA,5209.0,ZLO,1858.0,IAH,3550.0,Y,0,ERJ
61984,VW,1290.0,ZLO,1858.0,MEX,1824.0,NaN,0,ATR


In [99]:
DEST3=(lex_twotransfer["DEST"].unique())
DEST3
len(DEST3)

1853

In [100]:
lex_twotransfer_merged=pd.merge(lex_twotransfer, airports, left_on='SOURCE_ID', right_on='ID', how='left')
lex_twotransfer_merged=pd.merge(lex_twotransfer_merged, airports, left_on='DEST_ID', right_on='ID', how='left')
lex_twotransfer_merged=lex_twotransfer_merged.drop_duplicates(subset=['SOURCE_x', 'DEST'], keep='first')
lex_twotransfer_merged

,AIRLINE,AIRLINE_ID,SOURCE_x,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,ID_x,...,IATA_y,ICAO_y,LATITUDE_y,LONGITUDE_y,ALTITUDE_y,TIMEZONE_y,DST_y,TZ_y,TYPE_y,SOURCE
0,3M,20710.0,LWB,6958.0,ATL,3682.0,NaN,0,SF3,6958.0,...,ATL,KATL,33.636700,-84.428101,1026.0,-5.0,A,America/New_York,airport,OurAirports
1,UA,5209.0,LWB,6958.0,IAD,3714.0,Y,0,SF3,6958.0,...,IAD,KIAD,38.944500,-77.455803,312.0,-5.0,A,America/New_York,airport,OurAirports
2,3M,20710.0,MCN,3754.0,ATL,3682.0,NaN,0,SF3,3754.0,...,ATL,KATL,33.636700,-84.428101,1026.0,-5.0,A,America/New_York,airport,OurAirports
3,3M,20710.0,MCN,3754.0,MCO,3878.0,NaN,0,SF3,3754.0,...,MCO,KMCO,28.429399,-81.308998,96.0,-5.0,A,America/New_York,airport,OurAirports
4,3M,20710.0,MEI,4335.0,ATL,3682.0,NaN,0,SF3,4335.0,...,ATL,KATL,33.636700,-84.428101,1026.0,-5.0,A,America/New_York,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26252,UA,5209.0,ZIH,1856.0,IAH,3550.0,Y,0,ERJ,1856.0,...,IAH,KIAH,29.984400,-95.341400,97.0,-6.0,A,America/Chicago,airport,OurAirports
26255,4O,17885.0,ZLO,1858.0,MEX,1824.0,NaN,0,SU9,1858.0,...,MEX,MMMX,19.436300,-99.072098,7316.0,-6.0,S,America/Mexico_City,airport,OurAirports
26256,AM,321.0,ZLO,1858.0,LAX,3484.0,Y,0,73H,1858.0,...,LAX,KLAX,33.942501,-118.407997,125.0,-8.0,A,America/Los_Angeles,airport,OurAirports
26259,UA,5209.0,ZLO,1858.0,IAH,3550.0,Y,0,ERJ,1858.0,...,IAH,KIAH,29.984400,-95.341400,97.0,-6.0,A,America/Chicago,airport,OurAirports


In [43]:
DEST3=(lex_twotransfer["DEST"].unique())
lex_threetransfer=lex_twotransfer
for i in range(0,len(DEST3)):
    lex_threetransfer =lex_threetransfer.append(routes[(routes['SOURCE']==DEST3[i])])
lex_threetransfer=lex_threetransfer.drop_duplicates()
lex_threetransfer

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9
...,...,...,...,...,...,...,...,...,...
66062,YC,9828.0,SLY,6137.0,SVX,2975.0,NaN,0,CR2
66063,YC,9828.0,SLY,6137.0,TJM,4111.0,NaN,0,735 320 CR2
66064,YC,9828.0,SLY,6137.0,TQL,11436.0,NaN,0,L4T
66333,Z6,16120.0,UKS,7863.0,DME,4029.0,Y,0,ER4


In [44]:
DEST4=(lex_threetransfer["DEST"].unique())
DEST4
len(DEST4)

3056

In [45]:
DEST4=(lex_threetransfer["DEST"].unique())
lex_fourtransfer=lex_threetransfer
lex_fourtransfer
for i in range(0,len(DEST4)):
    lex_fourtransfer =lex_fourtransfer.append(routes[(routes['SOURCE']==DEST4[i])])
lex_fourtransfer=lex_fourtransfer.drop_duplicates()
lex_fourtransfer

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9
...,...,...,...,...,...,...,...,...,...
43845,NY,896.0,RKV,18.0,KUS,4248.0,NaN,0,DH8
19529,D2,20270.0,VUS,8924.0,CEE,6111.0,NaN,0,CR2
22156,DV,4840.0,SZI,9841.0,UKK,6086.0,NaN,0,AN4
39961,M9,3547.0,UDJ,6109.0,IEV,2944.0,NaN,0,A40


In [46]:
DEST5=(lex_fourtransfer["DEST"].unique())
DEST5
len(DEST5)

3300

In [47]:
DEST5=(lex_fourtransfer["DEST"].unique())
lex_fivetransfer=lex_fourtransfer
lex_fivetransfer
for i in range(0,len(DEST4)):
    lex_fivetransfer =lex_fivetransfer.append(routes[(routes['SOURCE']==DEST5[i])])
lex_fivetransfer=lex_fivetransfer.drop_duplicates()
lex_fivetransfer

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9
...,...,...,...,...,...,...,...,...,...
43845,NY,896.0,RKV,18.0,KUS,4248.0,NaN,0,DH8
19529,D2,20270.0,VUS,8924.0,CEE,6111.0,NaN,0,CR2
22156,DV,4840.0,SZI,9841.0,UKK,6086.0,NaN,0,AN4
39961,M9,3547.0,UDJ,6109.0,IEV,2944.0,NaN,0,A40


In [48]:
DEST6=(lex_fivetransfer["DEST"].unique())
DEST6
len(DEST6)

3300

In [49]:
x=(airlines["IATA"].unique())
x
len(x)

1121

In [51]:
routes

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
0,2B,410.0,AER,2965.0,KZN,2990.0,NaN,0,CR2
1,2B,410.0,ASF,2966.0,KZN,2990.0,NaN,0,CR2
2,2B,410.0,ASF,2966.0,MRV,2962.0,NaN,0,CR2
3,2B,410.0,CEK,2968.0,KZN,2990.0,NaN,0,CR2
4,2B,410.0,CEK,2968.0,OVB,4078.0,NaN,0,CR2
...,...,...,...,...,...,...,...,...,...
67658,ZL,4178.0,WYA,6334.0,ADL,3341.0,NaN,0,SF3
67659,ZM,19016.0,DME,4029.0,FRU,2912.0,NaN,0,734
67660,ZM,19016.0,FRU,2912.0,DME,4029.0,NaN,0,734
67661,ZM,19016.0,FRU,2912.0,OSS,2913.0,NaN,0,734


In [60]:
len(airports.ID.unique())

7184

In [63]:
airports[airports["ID"]==3682]

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
3467,3682,Hartsfield Jackson Atlanta International Airport,Atlanta,United States,ATL,KATL,33.6367,-84.428101,1026,-5.0,A,America/New_York,airport,OurAirports
